## vLLM setting

In [1]:
!nvidia-smi

import sys, torch
print("Python:", sys.version)
print("Torch:", torch.__version__)
print("Torch CUDA:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("Compute capability:", torch.cuda.get_device_capability(0))

Tue Jan  6 04:47:51 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ["HF_HOME"] = "/content/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/content/hf_cache"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
print("HF_HOME =", os.environ["HF_HOME"])

HF_HOME = /content/hf_cache


In [3]:
!pip -q uninstall -y transformers tokenizers
!pip -q install -U "transformers==4.45.2" "tokenizers==0.20.3" "accelerate>=0.33.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 27.1 MB/s eta 0:00:00


In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

In [1]:
%%capture
# 1) Cài PyTorch CUDA 12.1 (để tránh pip kéo nhầm torch CPU)
!pip -q install -U --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0

# 2) Cài vLLM (pin bản 0.6.6.post1) + các lib hay cần cho HF
!pip -q install -U \
  vllm==0.6.6.post1 \
  transformers accelerate sentencepiece safetensors tqdm

## Test vLLM

In [2]:
import transformers, vllm, torch

print("transformers:", transformers.__version__)
print("vllm:", vllm.__version__)
print("torch:", torch.__version__, "CUDA:", torch.version.cuda)

transformers: 4.57.3
vllm: 0.6.6.post1
torch: 2.5.1+cu124 CUDA: 12.4


In [3]:
!ls -lh /kaggle/working/Private_Test_ref.json \
        /kaggle/working/ground_truth_private_test.json \
        /kaggle/working/qwen_infer.py \
        /kaggle/working/evaluation.py

!mkdir -p /kaggle/working/predictions /kaggle/working/results /kaggle/working/logs
!ls -lh /kaggle/working/predictions /kaggle/working/results /kaggle/working/logs

ls: cannot access '/kaggle/working/Private_Test_ref.json': No such file or directory
ls: cannot access '/kaggle/working/ground_truth_private_test.json': No such file or directory
ls: cannot access '/kaggle/working/qwen_infer.py': No such file or directory
ls: cannot access '/kaggle/working/evaluation.py': No such file or directory
/kaggle/working/logs:
total 0

/kaggle/working/predictions:
total 0

/kaggle/working/results:
total 0


In [7]:
!cp /kaggle/input/logging/logger.py /kaggle/working/logger.py
!ls -l /kaggle/working/logger.py

-rw-r--r-- 1 root root 2622 Jan  6 04:58 /kaggle/working/logger.py


## Qwen inference

##### Few-shot

In [9]:
!python /kaggle/input/infer-of-qwen/qwen_infer.py \
  --input_file /kaggle/input/uit-viquad-2-0/Private_Test_ref.json \
  --output_file /kaggle/working/predictions_qwen_few.json \
  --model_name Qwen/Qwen2.5-7B-Instruct-AWQ \
  --mode few-shot \
  --max_new_tokens 32 \
  --temperature 0.0 \
  --top_p 0.7 \
  --max_context_chars 3500 \
  --batch_size 8 \
  --tensor_parallel_size 2 \
  --gpu_memory_utilization 0.9 \
  --log_dir /kaggle/working/logs

2026-01-06 05:01:30.343854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767675690.366534     417 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767675690.373290     417 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767675690.393115     417 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767675690.393143     417 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767675690.393147     417 computation_placer.cc:177] computation placer alr

## Evaluation

##### Few-shot

In [12]:
!python /kaggle/input/evaluate-main/evaluation.py \
  /kaggle/input/uit-viquad-2-0/ground_truth_private_test.json \
  /kaggle/working/predictions_qwen_few.json \
  --out-file /kaggle/working/results/evaluation_qwen_few.json \
  --log-dir /kaggle/working/logs

2026-01-06 06:11:53 - INFO - Logger initialized. Log file: /kaggle/working/logs/evaluation_20260106_061153.log
2026-01-06 06:11:53 - INFO - Loading dataset from /kaggle/input/uit-viquad-2-0/ground_truth_private_test.json
2026-01-06 06:11:53 - INFO - Loading predictions from /kaggle/working/predictions_qwen_few.json
2026-01-06 06:11:53 - INFO - No na_prob_file provided, using default 0.0 for all questions
2026-01-06 06:11:53 - INFO - Dataset contains 19 examples
2026-01-06 06:11:53 - INFO - Predictions contain 7301 entries
2026-01-06 06:11:54 - INFO - Evaluating on 3712 questions with predictions
2026-01-06 06:11:54 - INFO -   Has answer: 2596
2026-01-06 06:11:54 - INFO -   No answer: 1116
2026-01-06 06:11:54 - INFO - Evaluation Results:
2026-01-06 06:11:54 - INFO -   Exact Match: 1.24%
2026-01-06 06:11:54 - INFO -   F1 Score: 28.10%
2026-01-06 06:11:54 - INFO -   HasAns - Exact: 1.77%
2026-01-06 06:11:54 - INFO -   HasAns - F1: 40.18%
2026-01-06 06:11:54 - INFO -   NoAns - Exact: 0.00%